# Tutorial 08: list degrees of freedom associated to a restriction

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **penalty imposition of Dirichlet BCs**: this requires first the discretization of the associated homogeneous Neumann problem
$$
\text{find } u \in V \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V\\
$$
where
$$
V = H^1(\Omega),
$$
which we rewrite in matrix form as
$$ \text{find } x \in \mathbb{R}^n \text{ s.t. } A x = b,$$
where $A \in \mathbb{R}^{n \times n}$ and $b \in \mathbb{R}^n$ are the left-hand side and right-hand side of the discrete Neumann problem.

In order to impose Dirichlet boundary conditions, we solve the modified system
$$ \text{find } \tilde{x} \in \mathbb{R}^n \text{ s.t. } \tilde{A} \tilde{x} = \tilde{b},$$
where
$$\tilde{A} = A + P, \qquad \tilde{b} = b + q.$$

The penalty matrix $P$ is defined as
$$
P_{ij} =
\begin{cases}
\mu, & \text{if $i = j$, and $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
and the penalty vector $q$ as
$$
q_i = 
\begin{cases}
\mu g(\mathbf{x}_i), & \text{if $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
where $\mu \in \mathbb{R}$ is (large) scalar number, and $\mathbf{x}_i$ denotes the coordinate of the entity associated at DOF $i$.

The preferred way to impose non-homogeneous Dirichlet boundary conditions should still
either be through BlockDirichletBC objects (see tutorial 01) or lagrange multipliers
(see tutorial 03). This example is meant to show how to get the list of degrees of freedom
associated to a specific restriction, and how to perform local modifications to assembled
matrices.

In [ ]:
from numpy import arange, isclose, sin, sqrt, where
from petsc4py import PETSc
from ufl import grad, inner, Measure
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import assemble_scalar, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot
from multiphenics import *
from multiphenics.fem import block_assemble

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
facets_Gamma = where(boundaries.values == 1)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Helper functions

In [ ]:
def get_local_dofs(W, component):
    """
    Computes local dofs of W[component]. Returns two lists:
    * the first list stores local dof numbering with respect to W[component], e.g. to be used to fetch data
    from FEniCS solution vectors. Note that this list *neglects* restrictions. If interested in this output
    for restricted block function spaces, please see get_local_dofs_on_restriction.
    * the second list stores local dof numbering with respect to W, e.g. to be used to fetch data from
    multiphenics solution block_vector. Note that this list *considers* restrictions.
    """
    return (
        list(range(0, W[component].dofmap.index_map.block_size*W[component].dofmap.index_map.size_local)),
        list(range(0, W.block_dofmap.sub_index_map[component].block_size*W.block_dofmap.sub_index_map[component].size_local))
    )

In [ ]:
def get_local_dofs_on_restriction(W, component, restriction):
    """
    Computes dofs of W[component] which are on the provided restriction, which can be smaller or equal to the restriction
    provided at construction time of W (or it can be any restriction if W[component] is unrestricted). Returns two lists:
    * the first list stores local dof numbering with respect to W[component], e.g. to be used to fetch data
    from FEniCS solution vectors.
    * the second list stores local dof numbering with respect to W, e.g. to be used to fetch data from
    multiphenics solution block_vector.
    """
    # Extract unrestricted space associated to the provided component
    V = W.sub(component)
    # Prepare an auxiliary block function space, restricted on the boundary
    W_restricted = BlockFunctionSpace([V], restrict=[restriction])
    component_restricted = 0 # there is only one block in the W_restricted space
    # Get list of all local dofs on the restriction, numbered according to W_restricted. This will be a contiguous list
    # [1, 2, ..., # local dofs on the restriction]
    (_, restricted_dofs) = get_local_dofs(W_restricted, component_restricted)
    # Get the mapping of local dofs numbering from W_restricted[0] to V
    restricted_to_original = W_restricted.block_dofmap.block_to_original(component_restricted)
    # Get list of all local dofs on the restriction, but numbered according to V. Note that this list will not be
    # contiguous anymore, because there are DOFs on V other than the ones in the restriction (i.e., the ones in the
    # interior)
    original_dofs = [restricted_to_original[restricted] for restricted in restricted_dofs]
    # Get the mapping of local dofs numbering from V to W[b].
    original_to_block = W.block_dofmap.original_to_block(component)
    # Get list of all local dofs on the restriction, but numbered according to W. Note again that this list will not
    # be contiguous, and, in case of space W with multiple blocks, it will not be the same as original_dofs.
    block_dofs = [original_to_block[original] for original in original_dofs]
    return original_dofs, block_dofs

In [ ]:
def generate_penalty_system(W, component, restriction, penalty, g):
    """
    Generate matrix and vector to be added to the system to handle the penalty terms
    """
    (fenics_local_dofs, multiphenics_local_dofs) = get_local_dofs(W, component)
    (fenics_boundary_dofs, multiphenics_boundary_dofs) = get_local_dofs_on_restriction(W, component, restriction)
    fenics_interior_dofs = [dof for dof in fenics_local_dofs if dof not in fenics_boundary_dofs]
    # Assemble penalty matrix in a multiphenics matrix. Note that, as the matrix is assembled by multiphenics,
    # we will be using multiphenics_*_dofs variables, i.e. multiphenics numbering.
    p = [[u*v*dx]] # this u*v*dx form is not really used per se, just to allocated the correct sparsity pattern
    p = BlockForm2(p, [W, W])
    P = block_assemble(p)
    P.zeroEntries()
    for dof in multiphenics_boundary_dofs:
        P.setValuesLocal([dof], [dof], [penalty]) # set P_{ij} = penalty * \delta_{ij}, if i on boundary
    P.assemble()
    # Note that the for loop could have been replaced by
    #   P.mat().zeroRowsColumnsLocal(multiphenics_boundary_dofs, penalty)
    # We provide a manual version to show how to query petsc4py to manually change matrix entries (not necessarily on the
    # diagonal, but necessarily included in the sparsity pattern).
    #
    # Next, assemble the penalty vector. First, copy g and rescale by penalty, throwing away interior values.
    # Note that here we are using fenics_*_dofs variables, because g is a FEniCS vector.
    q_function = BlockFunction(W)
    g.vector.copy(result=q_function.sub(component).vector)
    q_function.sub(component).vector.scale(penalty)
    q_function.sub(component).vector.setValuesLocal(fenics_interior_dofs, [0.]*len(fenics_interior_dofs))
    q_function.sub(component).vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    q_function.apply("from subfunctions", component)
    # Return matrix and vector
    return P, q_function.block_vector

### Penalty imposition of Dirichlet BCs

In [ ]:
# Define a block function space
V = FunctionSpace(mesh, ("Lagrange", 2))
W = BlockFunctionSpace([V])

In [ ]:
# Define trial and test functions
(u, ) = BlockTrialFunction(W)
(v, ) = BlockTestFunction(W)

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: sin(3*x[0] + 1)*sin(3*x[1] + 1))
a = [[inner(grad(u), grad(v))*dx]]
f =  [v*dx                      ]
a = BlockForm2(a, [W, W])
f = BlockForm1(f, [W])
A = block_assemble(a)
F = block_assemble(f)

In [ ]:
# Add penalty terms
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
(P, Q) = generate_penalty_system(W, 0, dofs_V_Gamma, 1.e10, g)

In [ ]:
# Store options
options = PETSc.Options()
solver_parameters = {"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}
for k, v in solver_parameters.items():
    options.setValue("multiphenics_solve_" + k, v)
# Solve
U = BlockFunction(W)
solver = PETSc.KSP().create(W.mesh.mpi_comm())
solver.setOptionsPrefix("multiphenics_solve_")
solver.setFromOptions()
solver.setOperators(A + P)
solver.solve(F + Q, U.block_vector)
# Keep subfunctions up to date
U.apply("to subfunctions")

In [ ]:
plot(U[0])

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = DirichletBC(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = Function(V)
solve(a[0][0] == f[0], u_ex, bc_ex, petsc_options=solver_parameters)
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex), grad(u_ex))*dx)))
err_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex - U[0]), grad(u_ex - U[0]))*dx)))
print("Relative error is equal to", err_norm/u_ex_norm)
assert isclose(err_norm/u_ex_norm, 0., atol=1.e-10)